# SENTINEL HUD: Sistema de Seguridad Industrial con Vision Artificial

**Asignatura**: Vision por Computador Avanzada  
**Alumno**: Diego Sanchez  
**Fecha**: Febrero 2026

---

## 1. Definicion del Problema

La seguridad industrial es un desafio critico en entornos de construccion y manufactura. La falta de uso de Equipos de Proteccion Individual (EPIs) como cascos y chalecos reflectantes es una de las principales causas de accidentes laborales graves.

Los metodos tradicionales de supervision humana son propensos a errores, fatiga y falta de cobertura continua. Este proyecto, **SENTINEL HUD**, propone una solucion automatizada basada en Deep Learning (YOLOv5) para detectar en tiempo real el cumplimiento de normativas de seguridad, alertando proactivamente sobre trabajadores en riesgo (**Persona_Sin_EPI**).

## 2. Dataset y Preprocesamiento

Para este proyecto, se ha construido un dataset personalizado utilizando tecnicas de *scraping* automatizado (`dataset_builder.py`) para recopilar imagenes de entornos industriales reales.

**Clases del Dataset:**
1. `Casco` (Helmet)
2. `Chaleco` (Vest)
3. `Persona_Sin_EPI` (No PPE - Clase Critica)
4. `Peligro` (Danger Signs)

A continuacion, descomprimimos el paquete de entrega generado:

In [ ]:
import os
if os.path.exists('dataset_entrega.zip'):
    !unzip -q -o dataset_entrega.zip -d dataset_entrega
    print("[INFO] Dataset de entrega descomprimido.")
else:
    print("[WARN] Asegurate de subir 'dataset_entrega.zip' al entorno.")

## 3. Entrenamiento del Modelo (YOLOv5)

Utilizamos la arquitectura **YOLOv5s** (Small) por su eficiencia en inferencia de tiempo real, crucial para despliegue en Edge Devices.

**Configuracion:**
- **Batch Size**: 16
- **Epochs**: 50 (Suficiente para convergencia demostrativa)
- **Img Size**: 640px
- **Weights**: Pre-entrenados en COCO (Transfer Learning)

In [ ]:
# Instalacion de dependencias YOLOv5
if not os.path.exists('yolov5'):
    !git clone https://github.com/ultralytics/yolov5
    %cd yolov5
    %pip install -qr requirements.txt
else:
    %cd yolov5
    
import torch
from IPython.display import Image, display
print(f"Setup completo. Torch: {torch.__version__}")

In [ ]:
# Comando de Entrenamiento
!python train.py --img 640 --batch 16 --epochs 50 --data ../dataset_entrega/custom_data.yaml --weights yolov5s.pt --name sentinel_run --cache

## 4. Analisis de Resultados

El entrenamiento genera metricas clave para evaluar el desempeno del modelo:
- **Box Loss**: Error en la localizacion de objetos.
- **Objectness Loss**: Error en la deteccion de presencia de objetos.
- **mAP (Mean Average Precision)**: Metrica estandar de calidad en deteccion.

A continuacion, visualizamos el rendimiento del modelo entrenado:

In [ ]:
# Visualizacion de Metricas y Matriz de Confusion
import glob

# Buscar carpeta mas reciente para evitar errores de ruta
folders = glob.glob('runs/train/sentinel_run*')
if folders:
    latest_run = max(folders, key=os.path.getctime)
    print(f"Visualizando resultados de: {latest_run}")
    
    results_img = os.path.join(latest_run, 'results.png')
    confusion_img = os.path.join(latest_run, 'confusion_matrix.png')
    
    if os.path.exists(results_img):
        display(Image(filename=results_img, width=800))
    if os.path.exists(confusion_img):
        display(Image(filename=confusion_img, width=800))
else:
    print("Resultados no encontrados. Verifica la ejecucion del entrenamiento.")

## 5. Inferencia y Despliegue

Para superar los requisitos minimos del curso, SENTINEL HUD no se ha quedado en un notebook. El modelo entrenado (`best.pt`) ha sido desplegado en una **arquitectura de microservicios** utilizando:

1.  **DockerContainer**: Para aislar el entorno y asegurar reproducibilidad.
2.  **WebRTC**: Para permitir streaming de video de baja latencia desde el navegador hacia el contenedor.
3.  **Streamlit con CSS Personalizado**: Para una interfaz de usuario "Industrial Cyberpunk".

### Captura de Pantalla del Dashboard en Funcionamiento:

[INSERTAR CAPTURA DE PANTALLA DEL DASHBOARD AQUI]

> *La interfaz muestra el HUD superpuesto en tiempo real, con alertas rojas activas al detectar operarios sin equipo de proteccion.*